In [1]:
from pyspark.sql.functions import concat_ws, to_date, col
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer



In [2]:
spark = SparkSession.builder \
    .appName("HotelReservations_Transform") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/15 16:37:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.parquet("hotel_bookings_parquet")

In [4]:
df.show(10)
df.printSchema()

25/09/15 16:37:38 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------+-----------+---------+-----------------+------------------+------------------------+-------------------------+-----------------------+--------------------+------+--------+------+---------+-------+--------------+--------------------+-----------------+----------------------+------------------------------+------------------+------------------+---------------+------------+-----+-------+--------------------+---------------+------+---------------------------+-------------------------+------------------+-----------------------+-----------------+--------------------+------------+----------------+
|       hotel|is_canceled|lead_time|arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|stays_in_weekend_nights|stays_in_week_nights|adults|children|babies|     meal|country|market_segment|distribution_channel|is_repeated_guest|previous_cancellations|previous_bookings_not_canceled|reserved_room_type|assigned_room_type|booking_changes|deposit_type|agent

In [5]:
df = df.drop("name", "email", "phone-number", "credit_card", "reservation_status_date", "reservation_status", "required_car_parking_spaces", "days_in_waiting_list")
df.show(5)

+------------+-----------+---------+-----------------+------------------+------------------------+-------------------------+-----------------------+--------------------+------+--------+------+---------+-------+--------------+--------------------+-----------------+----------------------+------------------------------+------------------+------------------+---------------+------------+-----+-------+---------------+------+-------------------------+
|       hotel|is_canceled|lead_time|arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|stays_in_weekend_nights|stays_in_week_nights|adults|children|babies|     meal|country|market_segment|distribution_channel|is_repeated_guest|previous_cancellations|previous_bookings_not_canceled|reserved_room_type|assigned_room_type|booking_changes|deposit_type|agent|company|  customer_type|   adr|total_of_special_requests|
+------------+-----------+---------+-----------------+------------------+------------------------+----

In [6]:
df = df.withColumn(
    "arrival_date",
    to_date(
        concat_ws("-", col("arrival_date_year"), col("arrival_date_month"), col("arrival_date_day_of_month")),
        "yyyy-MMMM-d"
    )
)

df = df.drop("arrival_date_year", "arrival_date_month", "arrival_date_day_of_month")

df.show(5)

+------------+-----------+---------+------------------------+-----------------------+--------------------+------+--------+------+---------+-------+--------------+--------------------+-----------------+----------------------+------------------------------+------------------+------------------+---------------+------------+-----+-------+---------------+------+-------------------------+------------+
|       hotel|is_canceled|lead_time|arrival_date_week_number|stays_in_weekend_nights|stays_in_week_nights|adults|children|babies|     meal|country|market_segment|distribution_channel|is_repeated_guest|previous_cancellations|previous_bookings_not_canceled|reserved_room_type|assigned_room_type|booking_changes|deposit_type|agent|company|  customer_type|   adr|total_of_special_requests|arrival_date|
+------------+-----------+---------+------------------------+-----------------------+--------------------+------+--------+------+---------+-------+--------------+--------------------+-----------------+-

In [7]:
categorical_cols = ["hotel", "meal", "market_segment", "distribution_channel", "reserved_room_type", "assigned_room_type", "customer_type"]

for col_name in categorical_cols:
    indexer = StringIndexer(inputCol= col_name, outputCol= col_name + "_index")
    df = indexer.fit(df).transform(df)

df = df.drop("hotel", "meal", "market_segment", "distribution_channel", "reserved_room_type", "assigned_room_type", "customer_type")

df.show(5)

+-----------+---------+------------------------+-----------------------+--------------------+------+--------+------+-------+-----------------+----------------------+------------------------------+---------------+------------+-----+-------+------+-------------------------+------------+-----------+----------+--------------------+--------------------------+------------------------+------------------------+-------------------+
|is_canceled|lead_time|arrival_date_week_number|stays_in_weekend_nights|stays_in_week_nights|adults|children|babies|country|is_repeated_guest|previous_cancellations|previous_bookings_not_canceled|booking_changes|deposit_type|agent|company|   adr|total_of_special_requests|arrival_date|hotel_index|meal_index|market_segment_index|distribution_channel_index|reserved_room_type_index|assigned_room_type_index|customer_type_index|
+-----------+---------+------------------------+-----------------------+--------------------+------+--------+------+-------+-----------------+----

In [9]:
df = df.fillna({"children": 0, "babies": 0})

df.show(5)

+-----------+---------+------------------------+-----------------------+--------------------+------+--------+------+-------+-----------------+----------------------+------------------------------+---------------+------------+-----+-------+------+-------------------------+------------+-----------+----------+--------------------+--------------------------+------------------------+------------------------+-------------------+
|is_canceled|lead_time|arrival_date_week_number|stays_in_weekend_nights|stays_in_week_nights|adults|children|babies|country|is_repeated_guest|previous_cancellations|previous_bookings_not_canceled|booking_changes|deposit_type|agent|company|   adr|total_of_special_requests|arrival_date|hotel_index|meal_index|market_segment_index|distribution_channel_index|reserved_room_type_index|assigned_room_type_index|customer_type_index|
+-----------+---------+------------------------+-----------------------+--------------------+------+--------+------+-------+-----------------+----

In [ ]:
df = df.withColumn("total_stays", col("stays_in_weekend_nights") + col("stays_in_week_nights"))
df = df.withColumn("total_guests", col("adults") + col("children") + col("babies"))

df.show(100)

+-----------+---------+------------------------+-----------------------+--------------------+------+--------+------+-------+-----------------+----------------------+------------------------------+---------------+------------+-----+-------+------+-------------------------+------------+-----------+----------+--------------------+--------------------------+------------------------+------------------------+-------------------+-----------+------------+
|is_canceled|lead_time|arrival_date_week_number|stays_in_weekend_nights|stays_in_week_nights|adults|children|babies|country|is_repeated_guest|previous_cancellations|previous_bookings_not_canceled|booking_changes|deposit_type|agent|company|   adr|total_of_special_requests|arrival_date|hotel_index|meal_index|market_segment_index|distribution_channel_index|reserved_room_type_index|assigned_room_type_index|customer_type_index|total_stays|total_guests|
+-----------+---------+------------------------+-----------------------+--------------------+---